In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.proportion import proportions_ztest
from scipy import stats
from scipy.stats import chi2_contingency

%matplotlib inline

In [ ]:
# Load the data
base_url = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/main/5_6_eda_inf_stats_tableau/project/files_for_project/'

df_demo = pd.read_csv(base_url + 'df_final_demo.txt', delimiter=',')
df_client = pd.read_csv(base_url + 'df_final_experiment_clients.txt', delimiter=',')
df_web1 = pd.read_csv(base_url + 'df_final_web_data_pt_1.txt', delimiter=',')
df_web2 = pd.read_csv(base_url + 'df_final_web_data_pt_2.txt', delimiter=',')

df_web = pd.concat([df_web1, df_web2], ignore_index=True)

# Check available columns
print("df_client columns:", df_client.columns)

# Merge relevant data
df = df_client.merge(df_demo, on='client_id', how='left')
df = df.merge(df_web, on='client_id', how='left')

# Display columns of final dataframe
print("Final df columns:", df.columns)

df.head() 

In [ ]:
# Check unique steps
print(df['process_step'].unique())

# Assume last step is the final one
final_step = df['process_step'].dropna().unique().tolist()[-1]
print("Final step:", final_step)

# Tag completed users
df['completed'] = df['process_step'] == final_step

# Only keep one row per client, assuming final step means completed
df = df.groupby('client_id').agg({
    'Variation': 'first',
    'completed': 'max',
    'clnt_age': 'first',
    'clnt_tenure_yr': 'first',
    'gendr': 'first'
}).reset_index()

df['completed'] = df['completed'].astype(int)
df.head()

In [ ]:
completion_counts = df.groupby('Variation')['completed'].value_counts().unstack().fillna(0)
print(completion_counts)

success = np.array([completion_counts.loc['Test', 1], completion_counts.loc['Control', 1]])
nobs = np.array([completion_counts.loc['Test'].sum(), completion_counts.loc['Control'].sum()])

stat, pval = proportions_ztest(success, nobs)
print(f"Z-statistic: {stat:.4f}, p-value: {pval:.4f}")
if pval < 0.05:
    print("Reject H0: Significant difference in completion rate between groups.")
else:
    print("Fail to reject H0: No significant difference.") 

In [ ]:
test_rate = success[0] / nobs[0]
control_rate = success[1] / nobs[1]
diff = test_rate - control_rate

print(f"Test Rate: {test_rate:.4f}, Control Rate: {control_rate:.4f}, Difference: {diff:.4f}")

# Test if improvement is greater than 5%
threshold = 0.05
stat, pval = proportions_ztest(success, nobs, alternative='larger')

if diff >= threshold and pval < 0.05:
    print("Reject H0: The increase is significant and exceeds the 5% threshold.")
else:
    print("Fail to reject H0: Increase does not justify cost.")

In [ ]:
control_age = df[df['Variation'] == 'Control']['clnt_age'].dropna()
test_age = df[df['Variation'] == 'Test']['clnt_age'].dropna()

tstat, pval = stats.ttest_ind(control_age, test_age, equal_var=False)
print(f"T-statistic: {tstat:.4f}, p-value: {pval:.4f}")
if pval < 0.05:
    print("Reject H0: Average age differs between groups.")
else:
    print("Fail to reject H0: No significant difference in age.")


In [ ]:
sns.barplot(data=df, x='Variation', y='completed', estimator=np.mean, ci=95)
plt.title('Completion Rate by Group')
plt.ylabel('Completion Rate')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()


In [ ]:
sns.boxplot(data=df, x='Variation', y='clnt_age')
plt.title('Age Distribution by Group')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()


In [ ]:
completion_counts = df.groupby('Variation')['completed'].value_counts().unstack().fillna(0)
print(completion_counts)

success = np.array([completion_counts.loc['Test', 1], completion_counts.loc['Control', 1]])
nobs = np.array([completion_counts.loc['Test'].sum(), completion_counts.loc['Control'].sum()])

stat, pval = proportions_ztest(success, nobs)
print(f"Z-statistic: {stat:.4f}, p-value: {pval:.4f}")
if pval < 0.05:
    print("Reject H0: Significant difference in completion rate between groups.")
else:
    print("Fail to reject H0: No significant difference.")


In [ ]:
test_rate = success[0] / nobs[0]
control_rate = success[1] / nobs[1]
diff = test_rate - control_rate

print(f"Test Rate: {test_rate:.4f}, Control Rate: {control_rate:.4f}, Difference: {diff:.4f}")

# Test if improvement is greater than 5%
threshold = 0.05
stat, pval = proportions_ztest(success, nobs, alternative='larger')

if diff >= threshold and pval < 0.05:
    print("Reject H0: The increase is significant and exceeds the 5% threshold.")
else:
    print("Fail to reject H0: Increase does not justify cost.")


In [ ]:
control_age = df[df['Variation'] == 'Control']['clnt_age'].dropna()
test_age = df[df['Variation'] == 'Test']['clnt_age'].dropna()

tstat, pval = stats.ttest_ind(control_age, test_age, equal_var=False)
print(f"T-statistic: {tstat:.4f}, p-value: {pval:.4f}")
if pval < 0.05:
    print("Reject H0: Average age differs between groups.")
else:
    print("Fail to reject H0: No significant difference in age.")


In [ ]:
# Filter out unknown genders
df_gender = df[df['gendr'].isin(['M', 'F'])].copy()

# Group by gender
gender_completion = df_gender.groupby('gendr')['completed'].value_counts().unstack().fillna(0)
print(gender_completion)

# Z-test setup: Male vs Female
success = np.array([gender_completion.loc['M', 1], gender_completion.loc['F', 1]])
nobs = np.array([gender_completion.loc['M'].sum(), gender_completion.loc['F'].sum()])

stat, pval = proportions_ztest(success, nobs)
print(f"Z-statistic: {stat:.4f}, p-value: {pval:.4f}")

if pval < 0.05:
    print("Reject H0: Significant difference in completion rates between genders.")
else:
    print("Fail to reject H0: No significant gender-based difference in completion.")

In [ ]:
# Function to run and print results of gender z-test within each group
def gender_ztest_within_group(group_name):
    subset = df[(df['Variation'] == group_name) & (df['gendr'].isin(['M', 'F']))]
    completion = subset.groupby('gendr')['completed'].value_counts().unstack().fillna(0)

    success = np.array([completion.loc['M', 1], completion.loc['F', 1]])
    nobs = np.array([completion.loc['M'].sum(), completion.loc['F'].sum()])

    stat, pval = proportions_ztest(success, nobs)
    print(f"--- {group_name} Group ---")
    print(f"Z-statistic: {stat:.4f}, p-value: {pval:.4f}")
    if pval < 0.05:
        print("Reject H0: Significant gender difference in completion rate.")
    else:
        print("Fail to reject H0: No significant gender difference.")
    print()

# Run tests for each group
gender_ztest_within_group('Control')
gender_ztest_within_group('Test')


In [ ]:
df_gender = df[df['gendr'].isin(['M', 'F'])]
contingency = pd.crosstab(df_gender['gendr'], df_gender['completed'])
print("Contingency Table:\n", contingency)
chi2, pval, dof, expected = chi2_contingency(contingency)
print(f"Chi-Square Statistic: {chi2:.4f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {pval:.4f}")
if pval < 0.05:
    print("Reject H0: There is a significant relationship between gender and completion.")
else:
    print("Fail to reject H0: No significant relationship between gender and completion.")

In [3]:
## Hypothesis Testing
# H0=
#H1=

In [ ]:
def chi_square_gender_within(group_name):
    subset = df[(df['Variation'] == group_name) & (df['gendr'].isin(['M', 'F']))]
    contingency = pd.crosstab(subset['gendr'], subset['completed'])
    print(f"--- {group_name} Group ---")
    print("Contingency Table:\n", contingency)
    chi2, pval, dof, expected = chi2_contingency(contingency)
    print(f"Chi-Square Statistic: {chi2:.4f}")
    print(f"P-value: {pval:.4f}")
    if pval < 0.05:
        print("Reject H0: Significant relationship between gender and completion in this group.")
    else:
        print("Fail to reject H0: No significant gender relationship in this group.")
    print()

chi_square_gender_within('Control')
chi_square_gender_within('Test')

In [ ]:
sns.barplot(data=df[df['gendr'].isin(['M', 'F'])], x='gendr', y='completed', hue='Variation', estimator=np.mean, ci=95)
plt.title('Completion Rate by Gender within Control and Test Groups')
plt.ylabel('Completion Rate')
plt.xlabel('Gender')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()